In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

import time
from scipy.stats import uniform
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#getting the data
dataset = pd.read_csv('../input/laptop-price-predictor/laptop_pricing.csv')
print(dataset.Manufacturer.unique())

labels = ['Manufacturer']
for label in labels:
    dataset = pd.concat([dataset,pd.get_dummies(dataset[label], prefix=label)],axis=1)
    dataset.drop([label],axis=1, inplace=True)
dataset.head()

In [ ]:
# Lineaire correlatie tussen de features - heatmap
dataset.corr().abs()

# Visualiseer de onderlinge correlatiecoëfficiënten
f, ax = plt.subplots(figsize=(20, 20))
corr = dataset.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [ ]:
# Split up in features and targets. 
y = dataset['Price'].values
X = dataset.drop(['Price'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)   #Splitting train and test data

scaler = preprocessing.StandardScaler().fit(X_train)
# scaler = preprocessing.RobustScaler().fit(X_train)
# scaler = preprocessing.MinMaxScaler().fit(X_train)

# the data is fitted onto the train data, and applied to both train and test features
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Perform Grid-Search
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(3,10),
        'n_estimators': (10, 50, 100, 1000, 2000),
    },
    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_
print(best_params)

# Evaluatie van het model via de MAE, MSE en de R2-score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def evaluateModel(X_test, Y_test, model):

    y_predicted = model.predict(X_test)

    ## Mean Absolute Error
    MAE = mean_absolute_error(Y_test,y_predicted)
    print('MAE = ',MAE)

    ## Mean Squared Error
    MSE = mean_squared_error(Y_test,y_predicted)
    print('MSE = ',MSE)

    ## coefficient of determination = r2 score
    r2 = r2_score(Y_test,y_predicted)
    print('Test r2 score = ',r2)
    
    ## R2 score on train set
    y_predicted = model.predict(X_train)
    r2 = r2_score(y_train,y_predicted)
    print('Train r2 score = ',r2)
    
evaluateModel(X_test, y_test, grid_result) 



In [ ]:
import xgboost as xg 
  

# Instantiation 
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 1000, seed = 0) 
  
# Fitting the model 
xgb_r.fit(X_train, y_train) 
  
# Predict the model 
evaluateModel(X_test, y_test, xgb_r) 

We can see in the correlation matrix that dell laptops tend to be more pricey than others, also the speed of the processor doenst acount for to much. it is more the RAM of the computer, and the GB of th GPU that plays a key role. 

Furthermore we could add an extra collum, that instead of size of the storage medium we also have a parameter that says if we are using the SSD/ HDD/ ... 